<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

* Загрузим необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
import spacy


from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier



from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = stopwords.words('english')

pd.set_option('display.max_colwidth', 1000)
warnings.filterwarnings('ignore')

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seluy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\seluy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seluy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\seluy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


* Посмотрим наши данные

In [2]:
try:
    df = pd.read_csv("/datasets/toxic_comments.csv", index_col=[0])  
except:
    df = pd.read_csv("toxic_comments.csv", index_col=[0])
    
df.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков в данных нет, с типами тоже все в порядке

Соотношения классов:

In [4]:
df.toxic.value_counts()/df.shape[0]*100 

0    89.838787
1    10.161213
Name: toxic, dtype: float64

In [5]:
print(f"Количество дубликатов: {df.duplicated().sum()}")

Количество дубликатов: 0


Для дальнейшего использования 159272 очень большой датасет, поэтому сделаем sample.

In [6]:
sample_size = 40000
corpus = df.sample(n=sample_size,random_state=0).reset_index(drop=True)
print('соотношение классов в датасете corpus\n', corpus.toxic.value_counts()/corpus.shape[0]*100)

соотношение классов в датасете corpus
 0    89.78
1    10.22
Name: toxic, dtype: float64


* Очистим текст от лишних символов

In [7]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [8]:
corpus['text'] = corpus['text'].apply(clear_text) 

In [9]:
df.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


* Лемматизируем текст

Wordnet – это большая, свободно распространяемая и общедоступная лексическая база данных для английского языка с целью установления структурированных семантических отношений между словами.
Будем использовать  Wordnet Lemmatizer с соответствующим POS-тегом (part of speech - часть речи)

In [10]:
#Функция РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#Функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [27]:
%%time
new_corpus = []
nlp = spacy.load('en_core_web_sm')

for doc in tqdm(nlp.pipe(corpus['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(corpus['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))
    
corpus['lem_text'] = new_corpus  

100%|███████████████████████████████████████████████████████████████████████████| 40000/40000 [01:52<00:00, 355.18it/s]

CPU times: total: 1min 16s
Wall time: 1min 53s


* Разделим выборку на обучающую и тренировочную

In [28]:
#разделяю выборки в соотношении 80/20:
features = corpus['lem_text'] 
target = corpus['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)


<div class="alert alert-info"> <b>Комментарий студента:</b> Исправила</div>

In [29]:
for i in [features_train, target_train,features_test, target_test]:
    print(i.shape)

(32000,)
(32000,)
(8000,)
(8000,)


**Вывод**:
- Очистили тексты комментариев от ненужных знаков и леммализировали их
- Разбили данные на обучающуюи тестовую выборку в соотношении 80/20

## Обучение

* Логистическая регрессиия

In [30]:
from sklearn.pipeline import Pipeline


In [37]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} 

lr = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

lr.fit(features_train, target_train)

mts = lr.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,4))
print('при параметрах', lr.best_params_)


F1 логистической регрессии = 0.7534
при параметрах {'lr__C': 10, 'lr__class_weight': 'balanced', 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'newton-cg'}
CPU times: total: 6.27 s
Wall time: 47.1 s


* Дерево решений

In [38]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]), 
              'dtc__class_weight': (['balanced'])}

tree = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

tree.fit(features_train, target_train)

mts = tree.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,4))
print('при параметрах', tree.best_params_)

F1 дерева решений = 0.6158
при параметрах {'dtc__class_weight': 'balanced', 'dtc__max_depth': 23, 'dtc__random_state': 12345}
CPU times: total: 9.41 s
Wall time: 56.6 s


* CatBoostClassifier

In [39]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} #вот вообще не уверена, что class_weights тут нужен

cat = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

cat.fit(features_train, target_train)

mts = cat.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,4))
print('при параметрах', cat.best_params_)

F1 CatBoostClassifier = 0.7084
при параметрах {'cbc__class_weights': (1, 1), 'cbc__iterations': 200, 'cbc__verbose': False}
CPU times: total: 8min 16s
Wall time: 6min 9s


* RandomForestClassifier

In [40]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("rfc", RandomForestClassifier())])
    
parameters = {'rfc__n_estimators': ([x for x in range(10, 30)]),
              'rfc__random_state': ([12345]),
              'rfc__max_depth': ([x for x in range(1, 10)]),
              'rfc__criterion': (['entropy']),
              'rfc__class_weight': (['balanced'])}

rf = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

rf.fit(features_train, target_train)

mts = rf.cv_results_['mean_test_score']
rfc_train_f1 = max(mts)

print('F1 случайного леса =', round(rfc_train_f1,4))
print('при параметрах', rf.best_params_)

F1 случайного леса = 0.3533
при параметрах {'rfc__class_weight': 'balanced', 'rfc__criterion': 'entropy', 'rfc__max_depth': 9, 'rfc__n_estimators': 28, 'rfc__random_state': 12345}
CPU times: total: 40.3 s
Wall time: 4min 35s


* Сравнение моделей

In [41]:
#создаю сводную таблицу по показателям F1
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'RandomForestClassifier',
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    rfc_train_f1],
        }

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на обучающей выборке', ascending=False)

,F1 на обучающей выборке
LogisticRegression,0.753433
CatBoostClassifier,0.708362
DecisionTreeClassifier,0.615838
RandomForestClassifier,0.353299


Лучшей моделью я считаю логистическую регрессию

* Тестирование лучшей модели

In [36]:
predictions_test = lr.predict(features_test)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', lr_test_f1)

финальный F1 логистической регрессии = 0.7744733581164808


## Выводы

Мы проделали следующие шаги:

- загрузили данные и провели их предобработку - очистили текст, провели лемматизацию
- обучилм 4 модели LogisticRegression, DecisionTreeClassifier, CatBoostClassifier, RandomForestClassifier с разными гиперпараметрами и выборками, посмотрели результаты на валидационной выборке
- По показателю F1 лучшей выбрали LogisticRegression, провели тестирование и результатом оказалось 0.774, что удовлетворяет требованию задания
- Аутсайдером среди моделей стали RandomForestClassifier и DecisionTreeClassifier, так как дали наименьшее F1.

Итак, наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.774